In [13]:
import sys
sys.path.append(r'C:\Program Files\DIgSILENT\PowerFactory 2024\Python\3.11')
if __name__ == "__main__":
    import powerfactory as pf
app = pf.GetApplication()
if app is None:
    raise Exception('getting Powerfactory application failed')

In [14]:
#define project name and study case    
projName = 'BESS GFL-GFM_r03(3)'
study_case = '00_Comparison.IntCase'

#activate project
project = app.ActivateProject(projName)
proj = app.GetActiveProject()

#get the study case folder and activate project
oFolder_studycase = app.GetProjectFolder('study')
oCase = oFolder_studycase.GetContents(study_case)[0]
oCase.Activate()

#get load flow object and execute
oLoadflow=app.GetFromStudyCase('ComLdf') #get load flow object
oLoadflow.Execute() #execute load flow

0

In [15]:
#get the generators and their active/reactive power and loading
Generators = app.GetCalcRelevantObjects('*.ElmSym')
for gen in Generators: #loop through list
    name = getattr(gen, 'loc_name') # get name of the generator
    actPower = getattr(gen,'c:p') #get active power
    reacPower = getattr(gen,'c:q') #get reactive power
    genloading = getattr(gen,'c:loading') #get loading
    #print results
    print('%s: P = %.2f MW, Q = %.2f MVAr, loading = %.0f percent' %(name,actPower,reacPower,genloading))

print('-----------------------------------------')

#get the lines and print their loading
Lines=app.GetCalcRelevantObjects('*.ElmLne')
for line in Lines: #loop through list
    name = getattr(line, 'loc_name') # get name of the line
    value = getattr(line, 'c:loading') #get value for the loading
    #print results
    print('Loading of the line: %s = %.2f percent' %(name,value))

print('-----------------------------------------')

#get the buses and print their voltage
Buses=app.GetCalcRelevantObjects('*.ElmTerm')
for bus in Buses: #loop through list
    name = getattr(bus, 'loc_name') # get name of the bus
    amp = getattr(bus, 'm:u1') #get voltage magnitude
    phase = getattr(bus, 'm:phiu') #get voltage angle
    #print results
    print('Voltage at %s = %.2f pu %.2f deg' %(name,amp,phase))

Diesel Generator 1: P = 3.80 MW, Q = 0.00 MVAr, loading = 40 percent
Diesel Generator 2: P = 1.50 MW, Q = 0.00 MVAr, loading = 22 percent
Diesel Generator 1: P = 3.80 MW, Q = 0.00 MVAr, loading = 80 percent
Diesel Generator 2: P = 1.50 MW, Q = 0.00 MVAr, loading = 35 percent
-----------------------------------------
Loading of the line: L_10_11 = 0.12 percent
Loading of the line: L_1_2 = 0.37 percent
Loading of the line: L_2_3 = 0.37 percent
Loading of the line: L_3_4 = 0.44 percent
Loading of the line: L_3_8 = 0.37 percent
Loading of the line: L_4_11 = 0.22 percent
Loading of the line: L_4_5 = 0.34 percent
Loading of the line: L_5_6 = 0.11 percent
Loading of the line: L_6_7 = 0.06 percent
Loading of the line: L_7_8 = 0.09 percent
Loading of the line: L_8_9 = 0.11 percent
Loading of the line: L_9_10 = 0.12 percent
Loading of the line: L_10_11 = 0.09 percent
Loading of the line: L_1_2 = 0.37 percent
Loading of the line: L_2_3 = 0.37 percent
Loading of the line: L_3_4 = 0.64 percent
Load

In [16]:
# calculate initial conditions
oInit = app.GetFromStudyCase('ComInc') #get initial condition calculation object
oInit.Execute()

#run RMS-simulation
oRms = app.GetFromStudyCase('ComSim') #get RMS-simulation object
oRms.Execute()

0

In [17]:
import numpy as np
import pandas as pd

def getResults():    
    #get result file
    elmRes = app.GetFromStudyCase('*.ElmRes')    
    app.ResLoadData(elmRes)
  
    #Get number of rows and columns
    NrRow = app.ResGetValueCount(elmRes,0)
    
    #get objects of interest
    oSDG1 = app.GetCalcRelevantObjects('Diesel Generator 1.ElmSym')[0]
    oSDG2 = app.GetCalcRelevantObjects('Diesel Generator 2.ElmSym')[0]
    oBus1 = app.GetCalcRelevantObjects('T1.ElmTerm')[0]
    oLine3_4 = app.GetCalcRelevantObjects('L_3_4.ElmLne')[0]

    #Get index of variable of interest
    ColIndex_time = app.ResGetIndex(elmRes,elmRes,'b:tnow')
    ColIndex_ut = app.ResGetIndex(elmRes,oSDG1,'s:ut')
    ColIndex_P = app.ResGetIndex(elmRes,oSDG1,'s:P1')
    ColIndex_Q = app.ResGetIndex(elmRes,oSDG1,'s:Q1')
    ColIndex_speed = app.ResGetIndex(elmRes,oSDG1,'s:xspeed')
    ColIndex_u_bus1 = app.ResGetIndex(elmRes,oBus1,'m:u')
    ColIndex_loading_line_3_4 = app.ResGetIndex(elmRes,oLine3_4,'c:loading')
    
    #pre-allocate result variables
    result_time = np.zeros((NrRow,))
    result_ut = np.zeros((NrRow))
    result_P = np.zeros((NrRow))
    result_Q = np.zeros((NrRow))    
    result_speed = np.zeros((NrRow))    
    result_u_bus1 = np.zeros((NrRow))    
    result_loading_line_3_4 = np.zeros((NrRow))    
    
    #get results for each time step
    for i in range(NrRow):    
        result_time[i] = app.ResGetData(elmRes,i,ColIndex_time)[1]
        result_ut[i] = app.ResGetData(elmRes,i,ColIndex_ut)[1]
        result_P[i] = app.ResGetData(elmRes,i,ColIndex_P)[1]
        result_Q[i] = app.ResGetData(elmRes,i,ColIndex_Q)[1]       
        result_speed[i] = app.ResGetData(elmRes,i,ColIndex_speed)[1]       
        result_u_bus1[i] = app.ResGetData(elmRes,i,ColIndex_u_bus1)[1]       
        result_loading_line_3_4[i] = app.ResGetData(elmRes,i,ColIndex_loading_line_3_4)[1]       
    
    results = pd.DataFrame()
    results['time'] = result_time
    results['P'] = result_P
    results['Q'] = result_Q
    results['ut'] = result_ut
    results['speed'] = result_speed
    results['u_bus1'] = result_u_bus1
    results['loading_line_3_4'] = result_loading_line_3_4
    return results

#query results
RES = getResults()

In [19]:
RES[0:20]

,time,P,Q,ut,speed,u_bus1,loading_line_3_4
0,-1.0000,-1.0000,-1.0000,1.0,1.0,-1.0000,-1.0000
1,-0.9990,-0.9990,-0.9990,1.0,1.0,-0.9990,-0.9990
2,-0.9980,-0.9980,-0.9980,1.0,1.0,-0.9980,-0.9980
3,-0.9970,-0.9970,-0.9970,1.0,1.0,-0.9970,-0.9970
4,-0.9960,-0.9960,-0.9960,1.0,1.0,-0.9960,-0.9960
5,-0.9950,-0.9950,-0.9950,1.0,1.0,-0.9950,-0.9950
6,-0.9940,-0.9940,-0.9940,1.0,1.0,-0.9940,-0.9940
7,-0.9930,-0.9930,-0.9930,1.0,1.0,-0.9930,-0.9930
8,-0.9920,-0.9920,-0.9920,1.0,1.0,-0.9920,-0.9920
9,-0.9910,-0.9910,-0.9910,1.0,1.0,-0.9910,-0.9910
